In [1]:
import hemul
hemul.USE_FPGA=False
from hemul import heaan
from muxcnn.resnet_HEAAN import ResNetHEAAN
from muxcnn.utils import get_channel_last

Using CPU version HEAAN
Using CPU version HEAAN
Using CPU version HEAAN


## Torch model

In [2]:
#import torch
from muxcnn.models.ResNet20 import ResNet, BasicBlock
from muxcnn.utils import load_params, load_img

# 원본 모델
model = ResNet(BasicBlock,[1,1,1])
model.eval() ########## 필수! 
load_params(model, fn_param="./ResNet8.pt",device='cpu')

### 예제 이미지 load

In [3]:
img_tensor = load_img("./cute.jpg", hi=32, wi=32)

### Tensor 모양 변경
imgl = get_channel_last(img_tensor[0].detach().numpy())

# class에 default 값 저장되어있음 
#hi, wi, ch = imgl.shape
#ki = 1 # initial ki - multiplex parameter

# FHE

In [4]:
logp = 40
logq = 800
logn = 15

# Rotation 미리 준비 
rot_l = [2**i for i in range(15)]

# prepare for all rotations
rot_l = rot_l + [2**15-1, 
         2**15-33, 2**15-32, 2**15-31,
         2**15-17, 2**15-16, 2**15-15, 
         2**15-9,2**15-8, 2**15-7] + [3,5,7,9,15,17, 31, 33]

In [5]:
# Context & Evaluator
hec = heaan.HEAANContext(logn, logp, logq, boot=False, load_sk=True, rot_l=rot_l)

Initializing the scheme as the data owner
Loading a secret key from:  ./SecretKey.txt
loading secret key done.
HEAAN CKKS setup is ready 


In [6]:
fhemodel = ResNetHEAAN(model, hec, alpha=17)

# 이미지 암호화 
ctxt = fhemodel.pack_img_ctxt(img_tensor)

functions set
degrees = [15, 15, 15, 29, 29], margin = 0.03, eps = 0.02


## Inference

In [7]:
result = fhemodel(ctxt)

[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32, 3, 1, 3,  8)
[MultParConv] (ho,wo,co,ko,to,po) =(32,32,16, 1,16,  2)
[MultParConv] q = 2
ct_a 40 800 [-7.49019618e-01 -9.13725493e-01 -5.52941203e-01 -8.43137265e-01
  4.97824553e-09 -4.11764680e-01  9.80392722e-02 -8.50980400e-01
 -1.26850980e-09 -2.47058815e-01  1.29411819e-01 -6.78431386e-01
  6.04992100e-09  8.23529953e-02  2.94117689e-01  2.31372598e-01
 -2.87725793e-09  1.92156912e-01  1.52941225e-01  1.29411820e-01
  2.19880958e-10  2.78431419e-01 -5.52941204e-01 -5.45098064e-01
 -1.06690569e-09  1.05882405e-01  4.98039239e-01  4.50980432e-01
 -1.80061259e-10  7.41176484e-01  7.41176491e-01 -4.27450952e-01
  3.01359088e-09]
ct_d 40 720
temp  ----   40 800 [-1.71996313e-08 -8.50980368e-01 -4.58823520e-01 -7.49019615e-01
 -4.82304863e-10 -1.68627449e-01  9.01961321e-02 -8.11764717e-01
  3.52556270e-09 -1.45098127e-01  1.29411841e-01  3.17647108e-01
 -3.68219558e-08  1.13725534e-01 -5.05882363e-01  9.13725509e-01
  2.65182775e-08  3.098039

temp  ----   40 240 [-5.89595646e-05  2.42368015e-01  7.06355699e-01 -1.47980176e-06
  1.36623521e+00  6.61775770e-06 -5.59205278e-06  2.15879681e-01
 -2.91799419e-05 -2.09314521e-06  4.38122203e-06  2.32152442e-06
 -8.26906803e-05  1.78167364e-01  1.79165056e-02  9.43235380e-01
 -1.91436348e-05 -3.77664159e-07  4.43253943e-01  5.54353878e+00
  8.45830860e-01 -1.29291967e-05  1.31789897e+00  2.71853486e+00
  1.99518103e-06  2.62593053e-01 -5.87573928e-07  5.18599628e-06
 -6.97508946e-05  2.14933179e-01  6.66403747e-01  7.79179745e-02
 -1.08674480e-05]
temp  ----   40 240 [ 4.12942479e-05  2.39975585e-01  7.49254679e-01  1.05984081e-01
 -3.51810094e-06  1.27936305e-05 -8.70957837e-06  2.10624075e-01
  3.16783219e-01 -2.05987789e-06  3.49508538e-06  1.29212377e-06
  5.42888817e-01  8.63559242e-02  1.07422441e-01  1.29638353e+00
  7.77617412e-06  1.31960125e-02  4.40432260e-01  3.23480412e+00
 -4.30029312e-05  1.42742896e-01  1.93848711e+00  2.87723337e+00
  2.49024946e-01  1.17344875e-02

temp  ----   40 120 [ 2.10179231e-07  2.26761015e-05  2.86070458e-05  8.15836252e-07
  1.15475639e+00  4.36585307e-05 -4.13211109e-05 -3.92865334e-05
 -1.21805680e-05 -2.84101027e-06  3.27887832e-01  1.55533993e+00
 -3.51987419e-05  1.24987997e-06  1.13189367e+00  8.55500327e-07
  7.61025035e-05  4.64078396e-07 -1.01204032e-05  3.29955339e-06
  1.11068927e+00 -4.78570923e-06  2.68155809e-05 -2.13177007e-05
  2.90760081e-05  6.31978693e-01  3.47795974e-01  1.31697241e+00
 -3.20220894e-05  5.86196757e-01 -1.76715669e-05  1.87211467e+00
 -6.00924745e-05]
temp  ----   40 120 [ 8.64166202e-01 -2.91909069e-06  4.85155655e-05  3.05365268e-06
  1.68583914e+00 -2.32075280e-05  1.44958984e-05 -1.36612241e-05
  1.49727248e+00  8.07932421e-03  3.68721316e-01  1.04472277e-05
 -1.39648551e-04  1.49335507e-01 -3.97900486e-06  9.24507615e-02
  7.83984121e-01 -2.90940669e-06 -2.75361050e-05  2.67905334e-06
  1.42902132e+00  4.22910935e-06  2.65917945e-05 -4.47213368e-06
  1.47187163e+00  4.42430263e-01

temp  ----   40 800 [ 1.12593511e-09 -4.29689907e-09 -1.02708890e-09  7.79785077e-10
 -1.43365848e-08 -3.87371034e-09  9.68465385e-10  5.50527718e-11
  4.19468462e-10 -5.14730789e-10  1.12288101e-09 -1.98047562e-09
 -8.82572192e-11  6.33194182e-09  4.59671190e-10 -1.74613509e-09
  1.65089901e-09  6.99355978e-10  4.07062037e-09 -4.83106593e-09
  1.62154691e-09 -6.19660804e-09  6.95882134e-10  5.12001295e-09
 -6.20273663e-10  1.67051161e-09 -1.13162778e-08 -6.89446016e-09
  1.87080362e-09  4.92148517e-09 -1.07411026e-09  6.48604448e-09
 -1.39840546e-09]
temp  ----   40 800 [ 9.33718511e-07  5.51254840e-08  3.60357284e-09  5.43177704e-09
 -8.43489733e-08 -1.12624637e-08  4.83253757e-09 -9.25065926e-08
  1.20794747e-08  1.96046996e-08 -1.19068599e-09 -4.67945755e-08
  8.12109151e-08  2.76729041e-08  1.93170978e-09  2.97533921e-08
  1.21654903e-08  6.05373621e-09 -3.47442144e-09  8.35699936e-09
  8.04855969e-08  1.37095485e-08  1.30789458e-08  1.06226368e-07
 -4.20965157e-09  3.18637978e-08

CT_B 40 200
CT_B 40 200
CT_B 40 200
CT_B 40 200
CT_B 40 200
CT_B 40 200
CT_B 40 200


 ddddddd
[ 0.0408842   0.74541537 -1.28566138 ... -2.80161603 -1.68018269
 -0.47899709]
[MultParConv] (hi,wi,ci,ki,ti,pi) =(32,32,16, 1,16,  2)
[MultParConv] (ho,wo,co,ko,to,po) =(16,16,32, 2, 8,  4)
[MultParConv] q = 16
ct_a 40 800 [ 1.12593511e-09 -4.29689907e-09 -1.02708890e-09  7.79785077e-10
 -1.43365848e-08 -3.87371034e-09  9.68465385e-10  5.50527718e-11
  4.19468462e-10 -5.14730789e-10  1.12288101e-09 -1.98047562e-09
 -8.82572192e-11  6.33194182e-09  4.59671190e-10 -1.74613509e-09
  1.65089901e-09  6.99355978e-10  4.07062037e-09 -4.83106593e-09
  1.62154691e-09 -6.19660804e-09  6.95882134e-10  5.12001295e-09
 -6.20273663e-10  1.67051161e-09 -1.13162778e-08 -6.89446016e-09
  1.87080362e-09  4.92148517e-09 -1.07411026e-09  6.48604448e-09
 -1.39840546e-09]
ct_d 40 720
temp  ----   40 800 [ 1.12593511e-09 -4.29689907e-09 -1.02708890e-09  7.79785077e-10
 -1.43365848e-08 -3.87371034e-09  9.68465385e-

CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
ZZZZZZZZZZZZZZZZZZZZZZ  22
11111 <class.Ciphertext logp: 40 logq: 40 n: 32768>
APPR 15 40 40
[-0.68881863  0.53073417 -1.13288703 ... -1.05363362 -4.63413107
  0.24460095]
min max -8.285690508488203 6.423036911567315
bootstrap done
AFTER bootstrap 40 800
APPR 15 40 600
[-0.1423623   0.10981315 -0.23304121 ... -0.21695695 -0.84784487
  0.05067527]
min max -1.1577470943956443 1.0485452860720048
APPR 15 40 400
[-0.98884247  0.82659261 -1.17010586 ... -1.16528365 -0.9390066
  0.4188222 ]
min max -1.170108855439436 1.1701085714099189
APPR 29 40 200
[-1.00690527  0.99402947 -1.00991191 ... -1.00851234 -1.0013624
  1.00366823]
min max -1.016236996585753 1.0164876259893947
bootstrap done
AFTER bootstrap 40 800
APPR 29 40 560
[-0.99999928  1.00000512 -0.99999809 ... -1.00001022 -1.00001589
  0.99999518]
min max -1.0000326154349828 1

CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
CT_B 40 80
forward_bb
Shortcut <class.Ciphertext logp: 40 logq: 40 n: 32768>
ctxt <class.Ciphertext logp: 40 logq: 720 n: 32768>
<class.Ciphertext logp: 40 logq: 40 n: 32768> <class.Ciphertext logp: 40 logq: 40 n: 32768>
[-0.43477675 -0.43477682 -0.43477697 ... -0.43477698 -0.43477697
 -0.43477767]
[-3.65537264 -0.05986944 -3.72425224 ... -0.75035349  1.41215475
 -1.35217985]
APPR 15 40 40
[-0.43477675 -0.43477682 -0.43477697 ... -0.43477698 -0.43477697
 -0.43477767]
min max -0.4347806838793651 -0.4347729971793943
bootstrap done
AFTER bootstrap 40 800
APPR 15 40 600
[-0.0900073  -0.09000731 -0.09000734 ... -0.09000734 -0.09000734
 -0.09000749]
min max -0.09000810949938212 -0.09000651873815967
APPR 15 40 400
[-0.70439758 -0.70439747 -0.70439804 ... -0.7043983  -0.70439865
 -0.70439991]
min max -0.7044073678635248 -0.7043898300375251
AP

BB forward
결과 np 버전이랑 비교.  

문제 없으면 마지막 layer는 np 버전에서 가져온 뒤에 encrypt해서 실행 

In [ ]:
# 결과 확인 
dec = hec.decrypt(result)
print("Class =", np.argmax(dec[:10]))